In [ ]:
pwd()

: 

In [1]:
import os
os.chdir("../")
os.getcwd()

'c:\\Users\\hp\\Desktop\\CodeBase\\deep_CNN_Classifier'

In [ ]:
os.environ[MLFLOW_TRACKING_URI]="https://dagshub.com/Amir47/deep_CNN_Classifier.mlflow"
os.environ[MLFLOW_TRACKING_USERNAME]="Amir47"
os.environ[MLFLOW_TRACKING_PASSWORD]="dfee1f40f1d2823707d621541d08df26445c3a53"

In [15]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories, save_json

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
import tensorflow as tf

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [18]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self)-> EvaluationConfig:
        eval_config= EvaluationConfig(
            path_of_model= "artifacts/training/model.h5",
            training_data= "artifacts/data_ingestion/PetImages",
            mlflow_uri= "https://dagshub.com/Amir47/deep_CNN_Classifier.mlflow",
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
        return eval_config 

In [21]:
import mlflow
import mlflow.tensorflow
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
       model= self.load_model(self.config.path_of_model)
       self._valid_generator()
       self.score= model.evaluate(self.valid_generator)
   
    def save_score(self):
       scores= {"loss": self.score[0], "accuracy": self.score[1]}
       save_json(path=Path("scores.json"), data=scores)
       
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_params({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })
            #mlflow.tensorflow.log_model()

In [22]:
try:
    config = ConfigurationManager()
    val_config= config.get_validation_config()
    evaluation= Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 2670s 6s/step - loss: 1.9469 - accuracy: 0.8501
